In [3]:
from lib.acousticDataset import CochlScene

train_set = CochlScene(root_path='/root/data/CochlScene', mode='train')
wavform, label = train_set[0]

In [4]:
from lib.wavUtils import Components, AudioPadding, time_shift, AmplitudeToDB
from torchaudio import transforms as a_transforms
sample_rate=44100
n_mels=80
n_fft=2048
win_length=1024
hop_length=512
mel_scale='slaney'

tf_array = Components(transforms=[
    AudioPadding(sample_rate=sample_rate, random_shift=True, max_length=sample_rate*10),
    time_shift(shift_limit=.17, is_random=True, is_bidirection=True),
    a_transforms.MelSpectrogram(
        sample_rate=sample_rate, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length, win_length=win_length,
        mel_scale=mel_scale, f_min=20, f_max=sample_rate//2
    ),
    AmplitudeToDB(top_db=80., max_out=2.),
])
spectrogram = tf_array(wavform)
spectrogram.shape

torch.Size([1, 80, 862])